# 初始化設定

## 將程式下載到colab環境

In [1]:
!git clone https://github.com/lyndonzheng/TFill
%cd TFill

Cloning into 'TFill'...
remote: Enumerating objects: 248, done.
remote: Total 248 (delta 0), reused 0 (delta 0), pack-reused 248 (from 1)
Receiving objects: 100% (248/248), 29.34 MiB | 22.29 MiB/s, done.
Resolving deltas: 100% (30/30), done.
/content/TFill


## 上傳資料集

In [3]:
#將資料集複製到執行目錄下(由於dataset.py的載入方法基於執行目錄的相對路徑，因此在不更動程式的情況下這樣處理)
import shutil

In [ ]:
# 複製data資料集
source_path = "/content/drive/MyDrive/中興_深度學習/HW4/data"  # 請替換成你的data資料集的絕對路徑
target_path = "/content/TFill/mydata"        # Colab 中的執行目錄
shutil.copytree(source_path, target_path)
print(f"資料夾已從 {source_path} 複製到 {target_path}")


資料夾已從 /content/drive/MyDrive/中興_深度學習/HW4/data 複製到 /content/TFill/mydata


In [4]:
# 複製test資料集
source_path = "/content/drive/MyDrive/中興_深度學習/HW4/test"  # 請替換成你的test資料集的絕對路徑
target_path = "/content/TFill/mytest"        # Colab 中的執行目錄
shutil.copytree(source_path, target_path)
print(f"資料夾已從 {source_path} 複製到 {target_path}")

資料夾已從 /content/drive/MyDrive/中興_深度學習/HW4/test 複製到 /content/TFill/mytest


## 載入必要套件

In [5]:
!pip install dominate
!pip install ninja
!pip install visdom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408196 sha256=728eba08005033e53754151e6b8424557f71c29eeb4aae321b40ff47c35aefd9
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom


# Training PLACE2

## 處理資料集

In [ ]:
import os

def generate_image_list(folder_path):
    """
    Generate a .txt file containing all image filenames in a folder.

    Args:
        folder_path (str): Path to the folder containing images.
    """
    # Validate the folder path
    if not os.path.isdir(folder_path):
        raise ValueError(f"Provided path '{folder_path}' is not a valid directory.")

    # Get all image filenames in the folder
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif'}
    image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.splitext(f)[1].lower() in image_extensions]

    # Sort the filenames to ensure order
    image_files.sort()

    # Get the folder name and determine the output .txt path
    folder_name = os.path.basename(folder_path.rstrip(os.sep))
    parent_dir = os.path.dirname(folder_path)
    txt_path = os.path.join(parent_dir, f"{folder_name}.txt")

    # Write filenames to the .txt file
    with open(txt_path, 'w') as f:
        for image_file in image_files:
            f.write(image_file + '\n')

    print(f"Image list saved to: {txt_path}")

# Example usage:
# generate_image_list("/path/to/your/image_folder")


In [ ]:
generate_image_list("/content/TFill/mydata/place2_train")
generate_image_list("/content/TFill/mydata/train_mask")


Image list saved to: /content/TFill/mydata/place2_train.txt
Image list saved to: /content/TFill/mydata/train_mask.txt


## 設定參數

In [ ]:
#請用以下程式替換 dataloader.data_loader.py 91行到118行的elif內邏輯
elif mask_type == 3:
            # external mask from "Image Inpainting for Irregular Holes Using Partial Convolutions (ECCV18)"
            if self.opt.isTrain:
                mask_index = random.randint(0, self.mask_size-1)
            else:
                mask_index = item

            mask_transform = transforms.Compose(
                [
                    transforms.Resize([h, w])
                ]
            )
            mask_pil = Image.open(self.mask_paths[mask_index]).convert('L')
            mask = mask_transform(mask_pil)

            if self.opt.isTrain:
                mask = self._mask_dilation(mask)
            else:
                mask = np.array(mask) < 128
            mask = torch.tensor(mask).view(1, h, w).float()

            mask_pil.close()
            return mask, mask_type

## 開始訓練

嘗試訓練5000 iter

In [ ]:
#請用以下內容取代train.sh內的指令
python train.py \
--name place2 \
--img_file /content/TFill/mydata/place2_train.txt \
--mask_file /content/TFill/mydata/train_mask.txt \
--model tc \
--coarse_or_refine coarse \
--netT original \
--n_encoders 12 \
--n_decoders 0 \
--netD style \
--gpu_ids 0 \
--load_size 256 \
--fine_size 256 \
--batch_size 8 \
--nThreads 2 \
--display_port 8093 \
--attn_G \
--add_noise \
--display_ncols 0 \
--iter_count 0 \
--n_iter 4000 \
--n_iter_decay 1000 \
--print_freq 50 \
--save_iters_freq 2000 \
--save_latest_freq 2000 \
--checkpoints_dir /content/checkpoints \
--no_html \
--preprocess none \
--mask_type 3

In [ ]:
!sh ./scripts/train.sh

/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
--------------Options--------------
activation: leakyrelu
add_noise: True
attn_D: False
attn_E: False
attn_G: True
batch_size: 8
beta1: 0.5
beta2: 0.9
checkpoints_dir: /content/checkpoints
coarse_or_refine: coarse
continue_train: False
data_powers: 5
display_env: main
display_freq: 1000
display_id: None
display_ncols: 0
display_port: 8093
display_server: http://localhost
display_single_pane_ncols: 0
display_winsize: 256
down_layers: 4
dropout: 0.0
embed_dim: 5

## 繼續訓練

繼續訓練35000iter(共40000)

In [ ]:
--continue_train : 開關選項

In [ ]:
#請用以下內容取代train.sh內的指令
python train.py \
--name place2 \
--img_file /content/TFill/mydata/place2_train.txt \
--mask_file /content/TFill/mydata/train_mask.txt \
--model tc \
--coarse_or_refine coarse \
--netT original \
--n_encoders 12 \
--n_decoders 0 \
--netD style \
--gpu_ids 0 \
--load_size 256 \
--fine_size 256 \
--batch_size 8 \
--nThreads 2 \
--display_port 8093 \
--attn_G \
--add_noise \
--display_ncols 0 \
--iter_count 0 \
--n_iter 40000 \
--n_iter_decay 5000 \
--print_freq 50 \
--save_iters_freq 5000 \
--save_latest_freq 5000 \
--checkpoints_dir /content/checkpoints \
--no_html \
--preprocess none \
--mask_type 3 \
--continue_train

In [ ]:
!sh ./scripts/train.sh

/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
--------------Options--------------
activation: leakyrelu
add_noise: True
attn_D: False
attn_E: False
attn_G: True
batch_size: 8
beta1: 0.5
beta2: 0.9
checkpoints_dir: /content/checkpoints
coarse_or_refine: coarse
continue_train: True
data_powers: 5
display_env: main
display_freq: 1000
display_id: None
display_ncols: 0
display_port: 8093
display_server: http://localhost
display_single_pane_ncols: 0
display_winsize: 256
down_layers: 4
dropout: 0.0
embed_dim: 51

## 保存訓練後的模型

In [ ]:
#將目標路徑替換成你的儲存路徑
!cp -r /content/checkpoints/place2 /content/drive/MyDrive/中興_深度學習/HW4/5-TFill/place2

# Test place2

## 設定參數

In [ ]:
#請用以下程式替換 dataloader.data_loader.py 91行到118行的elif內邏輯
elif mask_type == 3:
            # external mask from "Image Inpainting for Irregular Holes Using Partial Convolutions (ECCV18)"
            if self.opt.isTrain:
                mask_index = random.randint(0, self.mask_size-1)
            else:
                mask_index = item

            mask_transform = transforms.Compose(
                [
                    transforms.Resize([h, w])
                ]
            )
            mask_pil = Image.open(self.mask_paths[mask_index]).convert('L')
            mask = mask_transform(mask_pil)

            if self.opt.isTrain:
                mask = self._mask_dilation(mask)
            else:
                mask = np.array(mask) < 128
            mask = torch.tensor(mask).view(1, h, w).float()

            mask_pil.close()
            return mask, mask_type

In [ ]:
test_data_names = ["FFHQ_test", "Shunghaitech_test", "celeba_test", "paris_test", "place2_chruch_indoor_test", "place2_chruch_ourdoor_test"]
test_mask_name="test_mask"

dir_root = "/content/TFill/mytest"
dir_output = "/content/results"
pretrain_root = "/content/checkpoints" # --checkpoints_dir

NAME = "place2"

## 開始測試

In [ ]:
for data_test in test_data_names:
  print("Processing: ", data_test)
  DIR_IMG = f"{dir_root}/{data_test}"
  DIR_MASK = f"{dir_root}/{test_mask_name}"
  OUTPUTS = f"{dir_output}/{data_test}"
  !python test.py --name $NAME --img_file $DIR_IMG --mask_file $DIR_MASK --results_dir $OUTPUTS --model tc --coarse_or_refine refine --gpu_id 0 --no_shuffle --batch_size 1 --preprocess none --mask_type 3 --load_size 256 --checkpoints_dir $pretrain_root --attn_G --add_noise

串流輸出內容已截斷至最後 5000 行。
    )
    (attn): TransformerEncoderLayer(
      (attn): MultiheadAttention(
        (dropout): Dropout(p=0.0, inplace=False)
        (to_q): Linear(in_features=512, out_features=512, bias=True)
        (to_k): Linear(in_features=512, out_features=512, bias=True)
        (to_v): Linear(in_features=512, out_features=512, bias=True)
        (to_out): Linear(in_features=512, out_features=512, bias=True)
      )
      (conv1): PartialConv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1))
      (conv2): PartialConv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1))
      (norm1): PixelwiseNorm()
      (norm2): PixelwiseNorm()
      (dropout): Dropout(p=0.0, inplace=False)
      (activation): GELU(approximate='none')
    )
    (block2): ResnetBlock(
      (norm1): PixelwiseNorm()
      (conv1): PartialConv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm2): PixelwiseNorm()
      (conv2): PartialConv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padd

## 保存測試結果

In [ ]:
for data_test in test_data_names:
  print("Saving: ", data_test)
  source_path = f"/content/results/{data_test}/{NAME}/test_latest/img_ref_out" # Colab 中的執行目錄
  dest_path = f"/content/drive/MyDrive/中興_深度學習/HW4/5-TFill/place2/result/{data_test}" # 請替換成你的預測結果的絕對路徑
  !cp -r $source_path $dest_path

Saving:  FFHQ_test
Saving:  Shunghaitech_test
Saving:  celeba_test
Saving:  paris_test
Saving:  place2_chruch_indoor_test
Saving:  place2_chruch_ourdoor_test


# Training celeba

## 處理資料集

In [ ]:
import os

def generate_image_list(folder_path):
    """
    Generate a .txt file containing all image filenames in a folder.

    Args:
        folder_path (str): Path to the folder containing images.
    """
    # Validate the folder path
    if not os.path.isdir(folder_path):
        raise ValueError(f"Provided path '{folder_path}' is not a valid directory.")

    # Get all image filenames in the folder
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif'}
    image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.splitext(f)[1].lower() in image_extensions]

    # Sort the filenames to ensure order
    image_files.sort()

    # Get the folder name and determine the output .txt path
    folder_name = os.path.basename(folder_path.rstrip(os.sep))
    parent_dir = os.path.dirname(folder_path)
    txt_path = os.path.join(parent_dir, f"{folder_name}.txt")

    # Write filenames to the .txt file
    with open(txt_path, 'w') as f:
        for image_file in image_files:
            f.write(image_file + '\n')

    print(f"Image list saved to: {txt_path}")

# Example usage:
# generate_image_list("/path/to/your/image_folder")


In [ ]:
generate_image_list("/content/TFill/mydata/celeba_train")
generate_image_list("/content/TFill/mydata/train_mask")


Image list saved to: /content/TFill/mydata/celeba_train.txt
Image list saved to: /content/TFill/mydata/train_mask.txt


## 設定參數

In [ ]:
#請用以下程式替換 dataloader.data_loader.py 91行到118行的elif內邏輯
elif mask_type == 3:
            # external mask from "Image Inpainting for Irregular Holes Using Partial Convolutions (ECCV18)"
            if self.opt.isTrain:
                mask_index = random.randint(0, self.mask_size-1)
            else:
                mask_index = item

            mask_transform = transforms.Compose(
                [
                    transforms.Resize([h, w])
                ]
            )
            mask_pil = Image.open(self.mask_paths[mask_index]).convert('L')
            mask = mask_transform(mask_pil)

            if self.opt.isTrain:
                mask = self._mask_dilation(mask)
            else:
                mask = np.array(mask) < 128
            mask = torch.tensor(mask).view(1, h, w).float()

            mask_pil.close()
            return mask, mask_type

## 開始訓練

訓練40000iter

In [ ]:
#請用以下內容取代train.sh內的指令
python train.py \
--name celeba \
--img_file /content/TFill/mydata/celeba_train.txt \
--mask_file /content/TFill/mydata/train_mask.txt \
--model tc \
--coarse_or_refine coarse \
--netT original \
--n_encoders 12 \
--n_decoders 0 \
--netD style \
--gpu_ids 0 \
--load_size 256 \
--fine_size 256 \
--batch_size 8 \
--nThreads 2 \
--display_port 8093 \
--attn_G \
--add_noise \
--display_ncols 0 \
--iter_count 0 \
--n_iter 35000 \
--n_iter_decay 5000 \
--print_freq 50 \
--save_iters_freq 5000 \
--save_latest_freq 5000 \
--checkpoints_dir /content/checkpoints \
--no_html \
--preprocess none \
--mask_type 3

In [ ]:
!sh ./scripts/train.sh

/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
--------------Options--------------
activation: leakyrelu
add_noise: True
attn_D: False
attn_E: False
attn_G: True
batch_size: 8
beta1: 0.5
beta2: 0.9
checkpoints_dir: /content/checkpoints
coarse_or_refine: coarse
continue_train: False
data_powers: 5
display_env: main
display_freq: 1000
display_id: None
display_ncols: 0
display_port: 8093
display_server: http://localhost
display_single_pane_ncols: 0
display_winsize: 256
down_layers: 4
dropout: 0.0
embed_dim: 5

## 保存訓練後的模型

In [ ]:
#將目標路徑替換成你的儲存路徑
!cp -r /content/checkpoints/celeba /content/drive/MyDrive/中興_深度學習/HW4/5-TFill/celeba/checkpoints

# Test celeba

## 設定參數

In [ ]:
#請用以下程式替換 dataloader.data_loader.py 91行到118行的elif內邏輯
elif mask_type == 3:
            # external mask from "Image Inpainting for Irregular Holes Using Partial Convolutions (ECCV18)"
            if self.opt.isTrain:
                mask_index = random.randint(0, self.mask_size-1)
            else:
                mask_index = item

            mask_transform = transforms.Compose(
                [
                    transforms.Resize([h, w])
                ]
            )
            mask_pil = Image.open(self.mask_paths[mask_index]).convert('L')
            mask = mask_transform(mask_pil)

            if self.opt.isTrain:
                mask = self._mask_dilation(mask)
            else:
                mask = np.array(mask) < 128
            mask = torch.tensor(mask).view(1, h, w).float()

            mask_pil.close()
            return mask, mask_type

In [ ]:
test_data_names = ["FFHQ_test", "Shunghaitech_test", "celeba_test", "paris_test", "place2_chruch_indoor_test", "place2_chruch_ourdoor_test"]
test_mask_name="test_mask"

dir_root = "/content/TFill/mytest"
dir_output = "/content/results"
pretrain_root = "/content/checkpoints" # --checkpoints_dir

NAME = "celeba"

## 開始測試

In [ ]:
for data_test in test_data_names:
  print("Processing: ", data_test)
  DIR_IMG = f"{dir_root}/{data_test}"
  DIR_MASK = f"{dir_root}/{test_mask_name}"
  OUTPUTS = f"{dir_output}/{data_test}"
  !python test.py --name $NAME --img_file $DIR_IMG --mask_file $DIR_MASK --results_dir $OUTPUTS --model tc --coarse_or_refine refine --gpu_id 0 --no_shuffle --batch_size 1 --preprocess none --mask_type 3 --load_size 256 --checkpoints_dir $pretrain_root --attn_G --add_noise

串流輸出內容已截斷至最後 5000 行。
    )
    (attn): TransformerEncoderLayer(
      (attn): MultiheadAttention(
        (dropout): Dropout(p=0.0, inplace=False)
        (to_q): Linear(in_features=512, out_features=512, bias=True)
        (to_k): Linear(in_features=512, out_features=512, bias=True)
        (to_v): Linear(in_features=512, out_features=512, bias=True)
        (to_out): Linear(in_features=512, out_features=512, bias=True)
      )
      (conv1): PartialConv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1))
      (conv2): PartialConv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1))
      (norm1): PixelwiseNorm()
      (norm2): PixelwiseNorm()
      (dropout): Dropout(p=0.0, inplace=False)
      (activation): GELU(approximate='none')
    )
    (block2): ResnetBlock(
      (norm1): PixelwiseNorm()
      (conv1): PartialConv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (norm2): PixelwiseNorm()
      (conv2): PartialConv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padd

## 保存測試結果

In [ ]:
for data_test in test_data_names:
  print("Saving: ", data_test)
  source_path = f"/content/results/{data_test}/{NAME}/test_latest/img_ref_out"  # 請替換成你的預測結果的絕對路徑
  dest_path = f"/content/drive/MyDrive/中興_深度學習/HW4/5-TFill/celeba/result/{data_test}"        # Colab 中的執行目錄
  !cp -r $source_path $dest_path

Saving:  FFHQ_test
Saving:  Shunghaitech_test
Saving:  celeba_test
Saving:  paris_test
Saving:  place2_chruch_indoor_test
Saving:  place2_chruch_ourdoor_test


# Complexity

## 設定參數

In [ ]:
#請用以下程式替換 dataloader.data_loader.py 91行到118行的elif內邏輯
elif mask_type == 3:
            # external mask from "Image Inpainting for Irregular Holes Using Partial Convolutions (ECCV18)"
            if self.opt.isTrain:
                mask_index = random.randint(0, self.mask_size-1)
            else:
                mask_index = item

            mask_transform = transforms.Compose(
                [
                    transforms.Resize([h, w])
                ]
            )
            mask_pil = Image.open(self.mask_paths[mask_index]).convert('L')
            mask = mask_transform(mask_pil)

            if self.opt.isTrain:
                mask = self._mask_dilation(mask)
            else:
                mask = np.array(mask) < 128
            mask = torch.tensor(mask).view(1, h, w).float()

            mask_pil.close()
            return mask, mask_type

In [6]:
test_data_names = ["FFHQ_test", "Shunghaitech_test", "celeba_test", "paris_test", "place2_chruch_indoor_test", "place2_chruch_ourdoor_test"]
test_mask_name="test_mask"

dir_root = "/content/TFill/mytest"
dir_output = "/content/results"
pretrain_root = "/content/checkpoints" # --checkpoints_dir

NAME = "celeba"

In [8]:
!pip install ptflops

In [ ]:
# model/tc_model.py 第84行後新增

        # Calculate FLOPs and parameters for each model
        from ptflops import get_model_complexity_info
        if not hasattr(self, 'flops_params_calculated'):  # Avoid recalculating multiple times
            print("Calculating FLOPs and Parameters...")
            with torch.cuda.device(0):  # Specify GPU device
                # For Encoder (netE)
                dummy_input_img = torch.randn(1, self.opt.img_nc, self.opt.fixed_size, self.opt.fixed_size).to(self.device)
                dummy_input_mask = torch.randn(1, self.opt.img_nc, self.opt.fixed_size, self.opt.fixed_size).to(self.device)
                flops_e, params_e = get_model_complexity_info(
                    self.netE, (dummy_input_img.shape[1], dummy_input_img.shape[2], dummy_input_img.shape[3]),
                    as_strings=True, verbose=True
                )
                print(f"Encoder (netE) FLOPs: {flops_e}, Parameters: {params_e}")

                # For Transformer (netT)
                dummy_input_img = torch.randn(512, 512, 1, 1).to(self.device)
                flops_t, params_t = get_model_complexity_info(
                    self.netT, (dummy_input_img.shape[1], dummy_input_img.shape[2], dummy_input_img.shape[3]),
                    as_strings=True, verbose=True
                )
                print(f"Transformer (netT) FLOPs: {flops_t}, Parameters: {params_t}")

                # For Generator (netG)
                dummy_input_img = torch.randn(512, 512, 3, 3).to(self.device)
                flops_g, params_g = get_model_complexity_info(
                    self.netG, (dummy_input_img.shape[1], dummy_input_img.shape[2], dummy_input_img.shape[3]),
                    as_strings=True, verbose=True
                )
                print(f"Generator (netG) FLOPs: {flops_g}, Parameters: {params_g}")

                # If Refiner exists
                if 'refine' in self.opt.coarse_or_refine:
                    flops_g_ref, params_g_ref = get_model_complexity_info(
                        self.netG_Ref, (dummy_input_img.shape[1], dummy_input_img.shape[2], dummy_input_img.shape[3]),
                        as_strings=True, verbose=True
                    )
                    print(f"Refiner (netG_Ref) FLOPs: {flops_g_ref}, Parameters: {params_g_ref}")
            self.flops_params_calculated = True


## 開始測試

In [20]:
data_test = "FFHQ_test"
print("Processing: ", data_test)
DIR_IMG = f"{dir_root}/{data_test}"
DIR_MASK = f"{dir_root}/{test_mask_name}"
OUTPUTS = f"{dir_output}/{data_test}"
!python test.py --name $NAME --img_file $DIR_IMG --mask_file $DIR_MASK --results_dir $OUTPUTS --model tc --coarse_or_refine refine --gpu_id 0 --no_shuffle --batch_size 1 --preprocess none --mask_type 3 --load_size 256 --checkpoints_dir $pretrain_root --attn_G --add_noise

Processing:  FFHQ_test
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
--------------Options--------------
activation: leakyrelu
add_noise: True
attn_D: False
attn_E: False
attn_G: True
batch_size: 1
checkpoints_dir: /content/checkpoints
coarse_or_refine: refine
data_powers: 5
display_env: main
display_id: None
display_port: 8092
display_server: http://localhost
display_single_pane_ncols: 0
display_winsize: 256
down_layers: 4
dropout: 0.0
embed_dim: 512
embed_type: learned
epoch: latest
eval: False
fine_siz